In [ ]:
import numpy as np
import os
import seaborn as sns

SIZE = 216

def fspecial_gauss(size: int, sigma: int) -> np.ndarray:
    """Function to mimic the 'fspecial' gaussian MATLAB function

        Parameters:
         size (int): size of square guassian kernel
         sigma (float): diameter of the kernel
    
        Returns:
         g (np.ndarray): gaussian kernel from [0, 1]
    """
    x, y = np.mgrid[-size//2 + 1:size//2 + 1, -size//2 + 1:size//2 + 1]
    g = np.exp(-((x**2 + y**2)/(2.0*sigma**2)))
    return g


def load_mosaic_feats(out_folder: str, depth) -> np.ndarray:
    _SIZE = 216
    """
    Loads the .npy subtile files in an output folder and mosaics the overlapping predictions
    to return a single .npy file of tree cover for the 6x6 km tile
    Additionally, applies post-processing threshold rules and implements no-data flag of 255
    
        Parameters:
         out_folder (os.Path): location of the prediction .npy files 
    
        Returns:
         predictions (np.ndarray): 6 x 6 km tree cover data as a uint8 from 0-100 w/ 255 no-data flag
    """
    
    # Generate the tiling schema
    x_tiles = [x for x in os.listdir(out_folder) if x.isnumeric()]
    x_tiles = [int(x) for x in x_tiles]
    max_x = np.max(x_tiles) + _SIZE
    for x_tile in x_tiles:
        y_tiles = [y[:-4] for y in os.listdir(out_folder + str(x_tile) + "/") if '.DS' not in y]
        y_tiles = [int(x.split("_")[0]) for x in y_tiles]
        #y_tiles = [int(y[:-4]) for y in os.listdir(out_folder + str(x_tile) + "/") if '.DS' not in y]
        if len(y_tiles) > 0:
            max_y = np.max(y_tiles) + _SIZE
        
    def _load_partial(out_folder, start, end, x_tiles = x_tiles, y_tiles = y_tiles, feats = False):
        # For predictions, load up a (SIZE, SIZE) array, * by 100
        # For features, load up a (SIZE, SIZE, 64) array and iterate thru 8 at a time
        #     due to memory constraints.
        # 
        MULT = 100

        n = end - start
        # 3 x 65 x 618 x 618,
            # 65, 618, 618 array
            # for i in range(0, 65, 8):
                # 25 x (8, 200, 200)
        predictions = np.full((n, max_x, max_y, len(x_tiles) * len(y_tiles)),
                              np.nan, dtype = np.float32)
        mults = np.full((1, max_x, max_y, len(x_tiles) * len(y_tiles)),
                        0, dtype = np.float32)
        i = 0
        for x_tile in x_tiles:
            y_tiles = [y[:-4] for y in os.listdir(out_folder + str(x_tile) + "/") if '.DS' not in y]
            y_tiles = [int(x.split("_")[0]) for x in y_tiles]
            for y_tile in y_tiles:
                output_file = out_folder + str(x_tile) + "/" + str(y_tile) + "_middle.npy"
                if os.path.exists(output_file):
                    prediction = np.load(output_file)
                    if prediction.shape[0] == 216:
                        GAUSS = 46
                    elif prediction.shape[0] > 200:
                        GUASS = 44
                    if prediction.shape[0] == 190:
                        GAUSS = 36
                    if prediction.shape[0] <= 150:
                        GAUSS = 30
                    if prediction.shape[0] <= 100:
                        GAUSS = 20
                    else:
                        GAUSS = 36
                    GAUSS = 36
                    #GAUSS = 46 if prediction.shape[0] > 150 else 5
                    if n > 1 or feats:
                        prediction = prediction[..., start:end]
                    else:
                        prediction[prediction < 255] = prediction[prediction < 255] * MULT
                    if (np.sum(prediction) < _SIZE*_SIZE*255) or depth > 1:
                        prediction = (prediction).T.astype(np.float32)
                        predictions[:, x_tile: x_tile+_SIZE, y_tile:y_tile + _SIZE, i] = prediction
                        fspecial_i = fspecial_gauss(_SIZE, GAUSS)
                        if depth == 1:
                            fspecial_i[prediction > 100] = 0.
                        mults[:, x_tile: x_tile+_SIZE, y_tile:y_tile + _SIZE, i] = fspecial_i # or 44

                    i += 1
            
        if depth > 1:
            predictions = predictions.astype(np.float32)
            mults = mults / np.sum(mults, axis = -1)[..., np.newaxis]
            predictions = np.nansum(predictions * mults, axis = -1)
            predictions = np.int16(predictions)
            return predictions
        else:
            mults = mults.squeeze()
            predictions = predictions.astype(np.float32)
            predictions = predictions.squeeze()

            ratios = np.zeros((predictions.shape[-1]), dtype = np.float32)
            mults[np.isnan(predictions)] = 0.
            try:
                for i in range(predictions.shape[-1]):
                    ratios[i] = calc_overlap(i, predictions)
                multipliers = np.median(ratios) / ratios
                multipliers[multipliers > 1.5] = 1.5
                for i in range(predictions.shape[-1]):
                    mults[..., i] *= multipliers[i]
            except:
                print("Skipping the weighted average due to cloud cover")

            predictions[predictions > 100] = np.nan
            
            mults = mults / np.sum(mults, axis = -1)[..., np.newaxis]

            out = np.copy(predictions)
            out = np.sum(np.isnan(out), axis = (2))
            n_preds = predictions.shape[-1]
            predictions = np.nansum(predictions * mults, axis = -1)
            predictions[out == n_preds] = np.nan
            predictions[np.isnan(predictions)] = 255.
            predictions = predictions.astype(np.uint8)

            predictions[predictions <= .15*MULT] = 0.        
            predictions[predictions > 100] = 255.
        
            return predictions
    
    output = np.full((depth, max_x, max_y),
                              0., dtype = np.int16)
    if depth == 1:
        output = _load_partial(out_folder, 1, 2)
    else:
        iters = np.arange(0, depth, 8)
        for i in iters:
            incrementer = np.min((depth - i, 8))
            print(f"The incrementer is {incrementer} for iter {i}")
            preds = _load_partial(out_folder, i, i + incrementer, feats = True)
            output[i:i + incrementer] = preds
    return output

In [ ]:
f = load_mosaic_feats('../project-monitoring/tiles/1656/1130/feats/', depth = 65)

In [ ]:
middle = np.load("../src/middle.npy")
#middle = np.moveaxis(middle, 1, 2)

sns.heatmap(middle[0].T)
print(middle.shape)

In [ ]:
import hickle as hkl
left = '../project-monitoring/tiles/1656/1130/raw/feats/1656X1130Y_feats.hkl'
left = hkl.load(left)
left = np.moveaxis(left, 1, 2)
sns.heatmap(left[0])
print(left.shape)

In [ ]:
import hickle as hkl
right =  '../project-monitoring/tiles/1657/1130/raw/feats/1657X1130Y_feats.hkl'
right = hkl.load(right)
right = np.moveaxis(right, 1, 2)

sns.heatmap(right[0])
print(right.shape)

In [ ]:
left_s = (lr.shape[2] - middle.shape[2]) // 2
print(left_s)

In [ ]:
lr = np.concatenate([left, right], axis = 2)
plt.figure(figsize =(20, 5))
#lr[1, :, left_s:left_s + 618] = lr[1, :, left_s:left_s + 618] + middle[1].T
#lr[1, :, left_s:left_s + 618] = lr[1, :, left_s:left_s + 618] / 2
#lr[1] /= 2
sns.heatmap(lr[1])

In [ ]:
sns.heatmap(middle[0].T)

In [ ]:
import hickle as hkl
def update_ard_tiles(x, y, m, base_path = '../project-monitoring/tiles/'):
    # Step 1 check if ARD exists
    # Step 2 update the 

    left_local_path = f'{base_path}{str(x)}/{str(y)}/raw/feats/{str(x)}X{str(y)}Y_feats.hkl'
    right_local_path = f'{base_path}{str(int(x) + 1)}/{str(y)}/raw/feats/{str(int(x) + 1)}X{str(y)}Y_feats.hkl'
    print("Loading original features")
    print(left_local_path)
    print(right_local_path)
    
    l = hkl.load(left_local_path) / 32768
    r = hkl.load(right_local_path) / 32768
    m = m / 32768
    l = np.moveaxis(l, 1, 2)
    r = np.moveaxis(r, 1, 2)
    m = np.moveaxis(m, 1, 2)
    #l = l.T
    #r = r.T
    #m = m.T    
    inp_mid_shape = m.shape[1]
    out_mid_shape = l.shape[1]
    middle_adjust = (inp_mid_shape - out_mid_shape) // 2
    m = m[:, middle_adjust:-middle_adjust, :]
    half = m.shape[1] // 2
    lsize = l.shape[1] - half
    rsize = lsize + half + half
    print("Concatenating three feature tiles ", l.shape, r.shape, m.shape)
    img = np.concatenate([l, r], axis = 1)
    print("Concatenated three feature tiles ", img.shape)
    sums = np.zeros((img.shape[0], img.shape[1]), dtype = np.float32)
    sums[:, :l.shape[0] // 2] = 1
    sums[:, l.shape[0] // 2:(l.shape[0] // 2)+half] += (1 - (np.arange(0, half, 1) / half))
    sums[:, (l.shape[0] // 2)+half:(l.shape[0] // 2)+half+half] += ((np.arange(0, half, 1) / half))
    sums[:, -(r.shape[0] // 2):] = 1.
    sums = sums[..., np.newaxis]
    sumsright = 1 - sums
    img[..., 1:] = img[..., 1:] * sums
    img[:, lsize:rsize, 1:] += (m * (1 - sums[:, lsize:rsize]))
    print("Blended three feature tiles, returning")
    #img[:, lsize:rsize] /= 2,# * (1 - sums[:, lsize:rsize]))
    #leftfile = img[:, :l.shape[1]]
    #rightfile = img[:, -r.shape[1]:]
    #hkl.dump(leftfile, left_local_path, mode='w', compression='gzip')
    #hkl.dump(rightfile, right_local_path,  mode='w', compression='gzip')
    #uploader.upload(bucket = 'tof-output', key = right_s3_path, file = right_local_path)
    #uploader.upload(bucket = 'tof-output', key = left_s3_path, file = left_local_path)
    return img, sums


def combine_resegmented_feats(x, y, m, indices, base_path = '../project-monitoring/tiles/'):
    # Step 1 check if ARD exists
    # Step 2 update the 

    left_local_path = f'{base_path}{str(x)}/{str(y)}/raw/feats/{str(x)}X{str(y)}Y_feats.hkl'
    right_local_path = f'{base_path}{str(int(x) + 1)}/{str(y)}/raw/feats/{str(int(x) + 1)}X{str(y)}Y_feats.hkl'
    print("Loading original features")
    print(left_local_path)
    print(right_local_path)
    
    l = hkl.load(left_local_path) / 32768
    r = hkl.load(right_local_path) / 32768
    m = m / 32768
    l = l.T
    r = r.T
    m = m.T    
    inp_mid_shape = m.shape[1]
    out_mid_shape = l.shape[1]
    middle_adjust = (inp_mid_shape - out_mid_shape) // 2
    m = m[:, middle_adjust:-middle_adjust, :]
    half = m.shape[1] // 2
    lsize = l.shape[1] - half
    rsize = lsize + half + half
    print("Concatenating three feature tiles ", l.shape, r.shape, m.shape)
    img = np.concatenate([l, r], axis = 1)
    print("Concatenated three feature tiles ", img.shape)
    sums = np.zeros((img.shape[0], img.shape[1]), dtype = np.float32)
    sums[:, :l.shape[0] // 2] = 1
    sums[:, l.shape[0] // 2:(l.shape[0] // 2)+half] += (1 - (np.arange(0, half, 1) / half))
    sums[:, (l.shape[0] // 2)+half:(l.shape[0] // 2)+half+half] += ((np.arange(0, half, 1) / half))
    sums[:, -(r.shape[0] // 2):] = 1.
    sums = sums[..., np.newaxis]
    sums = np.repeat(sums[..., np.newaxis], 65, axis = -1).squeeze()
    print(f"The sums are: {sums.shape}")
    print(f"The img shape is: {img.shape}")
    non_indices = [x for x in np.arange(0, 65) if x not in indices]
    print(f"The non-indices are: {non_indices}")
    sums[..., non_indices] = 1.
    #img = img * sums
    #img[:, lsize:rsize, 1:] += (m * (1 - sums[:, lsize:rsize, 1:])) # should be 0 for non-adj feats
    print("Blended three feature tiles")

    img = img.T
    img = np.int16(img * 32768)
    _left = img[:, :img.shape[1] // 2, :]
    _right = img[:, img.shape[1] // 2:, :]

    #img[:, lsize:rsize] /= 2,# * (1 - sums[:, lsize:rsize]))
    #leftfile = img[:, :l.shape[1]]
    #rightfile = img[:, -r.shape[1]:]
    print("Saving the updated feature tiles and continuing ", _left.shape, _right.shape)
    print(f"The left features are saved to: {left_local_path}")
    print(f"The right features are saved to: {right_local_path}")
   # hkl.dump(_left, left_local_path, mode='w', compression='gzip')
    #hkl.dump(_right, right_local_path,  mode='w', compression='gzip')
    #uploader.upload(bucket = 'tof-output', key = right_s3_path, file = right_local_path)
    #uploader.upload(bucket = 'tof-output', key = left_s3_path, file = left_local_path)
    return img, sums

#img2, sums = combine_resegmented_feats(1646, 1128, m, indices = np.arange(0, 65, 1))


def combine_resegmented_feats_north(x, y, m, indices, base_path = '../project-monitoring/tiles/'):
    # Step 1 check if ARD exists
    # Step 2 update the 

    left_local_path = f'{base_path}{str(x)}/{str(y)}/raw/feats/{str(x)}X{str(y)}Y_feats.hkl'
    right_local_path = f'{base_path}{str(int(x))}/{str(int(y) + 1)}/raw/feats/{str(int(x))}X{str(int(y + 1))}Y_feats.hkl'
    print("Loading original features")
    print(left_local_path)
    print(right_local_path)
    
    l = hkl.load(left_local_path) / 32768
    r = hkl.load(right_local_path) / 32768
    m = m / 32768
    l = l.T
    r = r.T
    m = m.T    
    inp_mid_shape = m.shape[0]
    out_mid_shape = l.shape[0]
    middle_adjust = (inp_mid_shape - out_mid_shape) // 2
    m = m[middle_adjust:-middle_adjust, :]
    half = m.shape[0] // 2
    lsize = l.shape[0] - half
    rsize = lsize + half + half
    print("Concatenating three feature tiles ", l.shape, r.shape, m.shape)
    img = np.concatenate([r, l], axis = 0)
    print("Concatenated three feature tiles ", img.shape)
    sums = np.zeros((img.shape[0], img.shape[1]), dtype = np.float32)
    print(sums.shape, l.shape[1] // 2)
    sums[:l.shape[1] // 2] = 1
    sums[l.shape[1] // 2:(l.shape[1] // 2)+half] += (1 - (np.arange(0, half, 1) / half))[:, np.newaxis]
    sums[(l.shape[1] // 2)+half:(l.shape[1] // 2)+half+half] += ((np.arange(0, half, 1) / half))[:, np.newaxis]
    rightend = (l.shape[1] // 2)+half+half
    print(rightend)
    sums[rightend:] = 1.
    sums = sums[..., np.newaxis]
    sums = np.repeat(sums[..., np.newaxis], 65, axis = -1).squeeze()
    print(f"The sums are: {sums.shape}")
    print(f"The img shape is: {img.shape}")
    non_indices = [x for x in np.arange(0, 65) if x not in indices]
    print(f"The non-indices are: {non_indices}")
    sums[..., non_indices] = 1.
    img = img * sums
    print(f"The image is: {img.shape}, the M is {m.shape}, the sums is {sums.shape}")
    img[lsize:rsize, :, 1:] += (m * (1 - sums[lsize:rsize, :, 1:])) # should be 0 for non-adj feats
    print("Blended three feature tiles")

    img = img.T
    img = np.int16(img * 32768)
    print(img.shape)
    _right = img[:, :, :img.shape[2] // 2]
    _left = img[:, :, -img.shape[2] // 2:]
    
    #leftfile = img[:, :, :l.shape[1]]
    #rightfile = img[:, :, -r.shape[1]:]
    print("Saving the updated feature tiles and continuing ", _left.shape, _right.shape)
    print(f"The left features are saved to: {left_local_path}")
    print(f"The right features are saved to: {right_local_path}")
   # hkl.dump(_left, left_local_path, mode='w', compression='gzip')
    #hkl.dump(_right, right_local_path,  mode='w', compression='gzip')
    #uploader.upload(bucket = 'tof-output', key = right_s3_path, file = right_local_path)
    #uploader.upload(bucket = 'tof-output', key = left_s3_path, file = left_local_path)
    return _left, _right

_left, _right = combine_resegmented_feats_north(1682, 1086, middle, indices = np.arange(0, 65, 1))

In [ ]:
sns.heatmap(_left[1])

In [ ]:
sns.heatmap(_right[1])

In [ ]:
sns.heatmap(sums[..., 0])

In [ ]:
import subprocess
right = '../project-monitoring/tiles/1674/1087/raw/feats/1674X1087Y_feats.hkl'
left = '../project-monitoring/tiles/1675/1087/raw/feats/1675X1087Y_feats.hkl'

right = '../project-monitoring/tiles/1646/1128/raw/feats/1646X1128Y_feats.hkl'
left = '../project-monitoring/tiles/1647/1128/raw/feats/1647X1128Y_feats.hkl'

right = '../project-monitoring/tiles/1672/1082/raw/feats/1672X1082Y_feats.hkl'
left = '../project-monitoring/tiles/1673/1082/raw/feats/1673X1082Y_feats.hkl'

#right = '../project-monitoring/tiles/1662/1068/raw/feats/1662X1068Y_feats.hkl'
#left = '../project-monitoring/tiles/1663/1068/raw/feats/1663X1068Y_feats.hkl'

subprocess.run(['aws', 's3', 'cp', right, 's3://restoration-monitoring/plantation-mapping/data/reseg_feats/'])

In [ ]:
import hickle as hkl
import seaborn as sns
import numpy as np
right = '../project-monitoring/tiles/1674/1087/raw/feats/1674X1087Y_feats.hkl'
left = '../project-monitoring/tiles/1675/1087/raw/feats/1675X1087Y_feats.hkl'

right = '../project-monitoring/tiles/1646/1128/raw/feats/1646X1128Y_feats.hkl'
left = '../project-monitoring/tiles/1647/1128/raw/feats/1647X1128Y_feats.hkl'

right = '../project-monitoring/tiles/1672/1082/raw/feats/1672X1082Y_feats.hkl'
left = '../project-monitoring/tiles/1673/1082/raw/feats/1673X1082Y_feats.hkl'

right = '../project-monitoring/tiles/1662/1068/raw/feats/1662X1068Y_feats.hkl'
left = '../project-monitoring/tiles/1663/1068/raw/feats/1663X1068Y_feats.hkl'

l = hkl.load(left)
l = np.moveaxis(l, 1, 2)
r = hkl.load(right)
r = np.moveaxis(r, 1, 2)
img = np.concatenate([r, l], axis = 2)

print(l.shape, r.shape)

import matplotlib.pyplot as plt
for i in range(0, 65, 1):
    plt.figure(figsize =(20, 5))
    sns.heatmap(img[i])
    plt.show()

In [ ]:
import numpy as np
import seaborn as sns
gain = np.load("../src/gaindatesarr.npy")
loss = np.load("../src/lossdate.npy")
dates = np.load("../src/dates.npy")
print(gain.shape, loss.shape, dates.shape)

from datetime import date
import datetime

d0 = date(2017, 1, 1)


In [ ]:
dates

In [ ]:
import rasterio as rs
#gtiff = rs.open('../src/example.tif').read(1)
multi_year = np.load("../src/multi_year.npy")
gtiff = np.median(multi_year, axis = 0)

baseline = multi_year[0]
sns.heatmap(gtiff)

In [ ]:
loss.shape

In [ ]:
gtiff[np.logical_or(gain > 0, loss > 0)] = baseline[np.logical_or(gain > 0, loss > 0)]
gtiff[gtiff > 100] = baseline[gtiff > 100]

In [ ]:
np.max(gain)

In [ ]:
5

In [ ]:
from matplotlib import pyplot as plt
newdata = np.copy(gtiff)
for i in range(1, 50):
    newdata[gain == i] = np.max(multi_year, axis = 0)[gain == i]
    newdata[loss == i] = np.min(multi_year, axis = 0)[loss == i]
    p = plt.figure(figsize=(10, 10))
    p = sns.heatmap(newdata, vmax = 100, cbar = False, cmap = 'Greens')
    p.tick_params(left=False, bottom=False)
    p.set(xticklabels=[], yticklabels=[])
    p.set_title(d0 + datetime.timedelta(days=int(dates[i])))
    plt.savefig(f'{str(i)}.png')
    plt.show()
    #plt.imsave(f'{str(i)}.png', newdata, cmap = 'Greens')